In [10]:
-- setup Jupyter notebook
:opt no-lint
:opt no-pager


-- necessary extensions & imports
{-# LANGUAGE OverloadedLabels #-}
{-# LANGUAGE TypeOperators #-}
{-# LANGUAGE DataKinds #-}
-- :set -F -pgmF=record-dot-preprocessor
import Data.Row.WebRecords
import Control.Lens



# Part 1. Declaring types of open records

Record types are parametrized by `Row`s 

In [2]:
:kind Rec

Rec :: Row * -> *

Basic operators for building a row:

In [3]:
:kind (.==)
:kind (.+)

(.==) :: forall k. Symbol -> k -> Row k

(.+) :: forall k. Row k -> Row k -> Row k

Example of a row:

In [4]:
type UserRow = "id" .== Int .+ "name" .== String .+ "friendIDs" .== [Int]

`(.+)` is commutative and aassociative: (plus demonstration of constraint chrecking)

In [5]:
{-# LANGUAGE TypeFamilies #-}
ok = () :: UserRow ~ ("name" .== String .+ ("friendIDs" .== [Int] .+ "id" .== Int)) => ()

Let's create a User :: Type

In [6]:
type User = Rec UserRow

:kind! User

User :: *
= Rec ('R '[ "friendIDs" ':-> [Int], "id" ':-> Int, "name" ':-> [Char]])

Internal structure, we should not create such types manually

# Part 2. Creating and accessing open records

We are using overloaded labels and old operators for creating records. Constraint `Forall l Unconstrained1` can be ignored here

In [7]:
:t (.+)
:t (.==)
:t (#x .==)

(.+) :: forall (l :: Row *) (r :: Row *). Forall l Unconstrained1 => Rec l -> Rec r -> Rec (l .+ r)

(.==) :: forall (l :: Symbol) a. KnownSymbol l => Label l -> a -> Rec ('R '[ l ':-> a])

(#x .==) :: forall a. a -> Rec ('R '[ "x" ':-> a])

All field labels and types are checked at compile time. Good enough error  messages:

Not all fields are initialized

In [8]:
bob :: User
bob = #id .== 12

: 

Typo in a field:

In [9]:
bob :: User
bob = #id .== 12
   .+ #friends .== [] 
   .+ #name .== "Bob"

: 

Wrong field type:

In [10]:
bob :: User
bob = #id .== 12
   .+ #name .== "Bob"
   .+ #friendIDs .== Nothing 

: 

So let's create a user:

In [11]:
bob :: User
bob = #name .== "Bob"
   .+ #id .== 12
   .+ #friendIDs .== [13, 14]

Autogenerated show and ToJSON/FromJSON instances don't care about order of fields:

In [12]:

import Data.Aeson
import Data.ByteString.Lazy as LBS
LBS.putStr . encode $ toJSON bob
bob

{"friendIDs":[13,14],"name":"Bob","id":12}

#friendIDs .== [13,14] .+ #id .== 12 .+ #name .== "Bob"

Field accessing via Lens:

In [13]:
bob ^. #id
view #name bob

12

"Bob"

In [14]:
f :: User -> (String, Int)
f u = (u ^. #name <> " #" <> show (u ^. #id), u ^. #id)

f bob

("Bob #12",12)

With record dot preprocessor, we can also write

```
f :: User -> (String, Int)
f u = (u.name <> " #" <> show (u.id), u.id)
```

but preprocessor is not stable, space-sensetive, and some advanced updates can be nicely expressed only with lens

# Part 3. Advanced updating
`Overloaded labels` allows us to use records as lenses for nested, polymorphic and monadic updates:

In [71]:
{-# LANGUAGE TypeApplications #-}
import Data.Row.Records
z :: User
z = default' @Read (read "")

z

: 

# Part 4. Changing a structure

Let's create a function that adds a field `name` to record:

In [29]:
giveName s obj = (#name .== s) .+ obj

:t giveName

giveName :: forall a (r :: Row *). a -> Rec r -> Rec ('R '[ "name" ':-> a] .+ r)

`.+` in inferred result type is a type family that can raise a type error if such field already exists:

In [30]:
thing = #id .== 124 
     .+ #struct .== (#aaa .== "aaa" .+ #bbb .== "bbb")
     .+ #name .== "Ken" 

thing

:t giveName "The thing" thing
giveName "The thing" thing

#id .== 124 .+ #name .== "Ken" .+ #struct .== (#aaa .== "aaa" .+ #bbb .== "bbb")

giveName "The thing" thing :: forall a. Num a => Rec ('R (("id" ':-> a) : (TypeError ...)))

: 

We can rename a field in struct to fix this

In [17]:
giveName "The thing" $ rename #name #oldName thing  

#id .== 124 .+ #name .== "The thing" .+ #oldName .== "Ken" .+ #struct .== (#aaa .== "aaa" .+ #bbb .== "bbb")

Also we can simply drop name field from old structure: 

In [22]:
giveName "The thing" $ thing .- #name 

#id .== 124 .+ #name .== "The thing" .+ #struct .== (#aaa .== "aaa" .+ #bbb .== "bbb")

Suppose we have structure like this:

In [18]:
struct = #user .== bob .+ #thing .==  thing

LBS.putStr . encode $ toJSON struct

{"thing":{"struct":{"bbb":"bbb","aaa":"aaa"},"name":"Ken","id":124},"user":{"friendIDs":[13,14],"name":"Bob","id":12}}

We can split a record to two parts, or restrict to subset. Via lens we can focus on subrecords to operate on them

In [59]:
import Data.Row.Records (split)

type UserInfo = Rec ("name" .== String .+ "id" .== Int)
makeUser :: UserInfo -> [Int] -> User
makeUser u friends = u .+ #friendIDs .== friends

userInfo :: User -> UserInfo
userInfo = restrict

splitUser :: User -> (UserInfo, [Int])
splitUser u = let (a, b) = Data.Row.Records.split u in (a, b ^. #friendIDs)

splitUser bob

(#id .== 12 .+ #name .== "Bob",[13,14])

In [36]:
import Data.Row.Records (restrict)
struct2 :: Rec ("thing" .== Rec ("name" .== String) .+ "user" .== Rec ("name" .== String))
struct2 = struct & #thing %~ restrict
                 & #user %~ restrict

struct2

#thing .== (#name .== "Ken") .+ #user .== (#name .== "Bob")

# Part 5. Functions to and from record structures

`r .! "a" ` is either value of #a in r or TypeError

In [9]:
{-# LANGUAGE FlexibleContexts #-}
f x = x ^. #aaa == 1

f (#aaa .== 12)

False